<a href="https://colab.research.google.com/github/kookeej/DILAB/blob/main/7.1-7.7/lab04-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab04-1 - Multivariable Linear Regression
===

In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
x_train = torch.FloatTensor([[73, 80, 75],
                            [93, 88, 93],
                            [89, 91, 90],
                            [96, 98, 100],
                            [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

만약 dataset이 많지 않다면 H(x1, x2, x3) = w1x1 + w2x2 + w3x3 + b 와 같은 형태로 식을 만들 수 있을 것이다.     
![](https://render.githubusercontent.com/render/math?math=H%28x_1%2C%20x_2%2C%20x_3%29%20%3D%20x_1w_1%20%2B%20x_2w_2%20%2B%20x_3w_3%20%2B%20b&mode=display)    

하지만 만약 데이터가 너무 많아진다면 식을 일일이 하나씩 적는 것은 불가능할 것이다. 따라서 우리는 ```matmul```함수를 사용한다.    
```matmul```은 Matrix multiplication이다. 이 연산을 사용해서 **더 간결하고 x의 길이 바뀌어도 코드를 바꿀 필요도 없고 속도도 더 빠른** hypothesis를 만들 수 있다.

In [ ]:
hypothesis = x_train.matmul(W) + b

* cost function은 기존의 식과 동일하다.            
![cost_function](https://render.githubusercontent.com/render/math?math=cost%28W%2C%20b%29%20%3D%20%5Cfrac%7B1%7D%7Bm%7D%20%5Csum%5Em_%7Bi%3D1%7D%20%5Cleft%28%20H%28x%5E%7B%28i%29%7D%29%20-%20y%5E%7B%28i%29%7D%20%5Cright%29%5E2&mode=display)


In [ ]:
cost = torch.mean((hypothesis - y_train) ** 2)

아래는 전체 코드이다.

In [8]:
W = torch.zeros((3,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = torch.optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    hypothesis = x_train.matmul(W) + b
    cost = torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7967, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936096
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371063
Epoch    6/20 hypothesis: tensor([148.1035, 178.0143, 175.3980, 191.0042, 135.7812]) Cost: 29.758249
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8509, 193.6753, 137.6805]) Cost: 10.445267
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391237
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493121
Epoch   10/20 hypo

여기서 W와 b를 직접 작성하는 것은 모델이 커질 수록 번거로운 일이 된다. 그래서 ```nn.Module```을 사용하여 모델을 생성한다.

* ```nn.Linear(3, 1)```: 입력 차원 3, 출력 차원 1
* Hypothesis 계산은 forward()에서
* Gradient 계산은 pytorch가 backward()로 알아서 해준다.

In [13]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

* F.mse_loss: cost function을 제공한다.쉽게 다른 loss와 교체가 가능하다.    

전체 코드를 살펴보면 아래와 같다.

In [16]:
model = MultivariateLinearRegressionModel()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    Hypothesis = model(x_train)
    cost = F.mse_loss(Hypothesis, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 7889.118164
Epoch    1/20 Cost: 2473.778809
Epoch    2/20 Cost: 776.357422
Epoch    3/20 Cost: 244.304977
Epoch    4/20 Cost: 77.534653
Epoch    5/20 Cost: 25.260645
Epoch    6/20 Cost: 8.875295
Epoch    7/20 Cost: 3.738957
Epoch    8/20 Cost: 2.128695
Epoch    9/20 Cost: 1.623652
Epoch   10/20 Cost: 1.465010
Epoch   11/20 Cost: 1.414954
Epoch   12/20 Cost: 1.398930
Epoch   13/20 Cost: 1.393594
Epoch   14/20 Cost: 1.391585
Epoch   15/20 Cost: 1.390650
Epoch   16/20 Cost: 1.390025
Epoch   17/20 Cost: 1.389486
Epoch   18/20 Cost: 1.389006
Epoch   19/20 Cost: 1.388525
Epoch   20/20 Cost: 1.388056
